# Debugging ResNet

Notebook walking through some best practice unit tests for performing sequentially in order to debug a misbehaving black box model.

Created by: Jacob A Rose  
Created On: Tuesday, March 16th, 2021
# 

In [4]:
import torch 
a = torch.ones(5)
a.requires_grad = True

b = 2*a

b.retain_grad()   # Since b is non-leaf and it's grad will be destroyed otherwise.

c = b.mean()

c.backward()

print('a =', a.grad,',\nb =', b.grad)

a = tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) ,
b = tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [2]:
b

tensor([2., 2., 2., 2., 2.], grad_fn=<MulBackward0>)

In [5]:
# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print(x))  

b.mean().backward() 


print('a =', a.grad,',\nb =', b.grad)

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
a = tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) ,
b = tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
